# Monte Carlo Tree Search - Andre Nguyen

In [8]:
from __future__ import division
import numpy as np
from numpy.random import rand
import math
from random import randint
import itertools
import matplotlib.pyplot as plt
import copy
from multiprocessing import Pool
%matplotlib inline

In [9]:
# Ken's break ties function.
def argmax_breaking_ties_randomly(x):
    max_value = np.max(x)
    indices_with_max_value = np.flatnonzero(x == max_value)
    return np.random.choice(indices_with_max_value)

## Angela's Code

In [10]:
class ConnectN:
    
    def __init__(self, grid_size, n):
        self.n = n
        self.grid_size = grid_size
        self.grid = np.zeros([grid_size,grid_size])
        self.finished = 0
        self.turn_num = 0
        
    def reset(self):
        self.__init__(self.grid_size, self.n)

    def check_win(self, col, row, player):
        for i in range(0, self.n):
            if sum(self.grid[col, row - i:row - i + self.n]) == self.n*player:
                self.finished = 1
                return 1
            if sum(self.grid[col - i: col - i + self.n, row]) == self.n*player:
                self.finished = 1
                return 1
            if col - i >= 0 and col - i + self.n - 1 < self.grid_size and row - i >= 0 and row - i + self.n - 1 < self.grid_size:
                if sum([self.grid[col - i + x, row - i + x] for x in range(0, self.n)]) == self.n*player:
                    self.finished = 1
                    return 1
            if col - i >= 0 and col - i + self.n - 1 < self.grid_size and row + i >= self.n - 1 and row + i < self.grid_size:
                if sum([self.grid[col - i + x, row + i - x] for x in range(0, self.n)]) == self.n*player:
                    self.finished = 1
                    return 1
        return 0

    def move(self, col, player):
        
        self.turn_num += 1
        
        if self.finished == 1:
            return 1, 50
        sum_col = np.sum([abs(x) for x in self.grid[col]])
        if sum_col == self.grid_size:
            return -1, -1
        self.grid[col, sum_col] = player
        if self.check_win(col, sum_col, player) == 1:
            return 1, 50
        return 0, 0
    
    def turn(self):
        """
        Returns which player's turn it is. First turn is player 1, second turn is player -1.
        """
        if self.turn_num%2 == 0:
            return 1
        else:
            return -1
        
    def next_possible_moves(self):
        """
        Returns array of possible columns for a next move
        """
        columns = []
        
        for i in xrange(0, self.grid_size):
            if (0 in self.grid[i]):
                columns.append(i)
                
        return columns
    
    def all_tokens_placed(self):
        """
        Returns location of all tokens (column, row) that have been placed
        """
        all_tokens = []
        
        for col in xrange(0, self.grid_size):
            for row in xrange(0, self.grid_size): 
                if self.grid[col][row] != 0:
                    all_tokens.append({"location": [col, row], "player": self.grid[col][row]})
                    
        return all_tokens
    
    def is_empty(self, col, row):
        return self.grid[col][row] == 0
    
    def print_grid(self):
        print(np.rot90(self.grid))

In [11]:
def run_trial(agent, MIN_ITERATIONS, MIN_EPISODES, player):
    """
    Runs the ConnectN Simulator for one player given an agent and number of iterations and episodes
    """
    
    rewards_by_iteration = []
    rewards_by_episode = []
    cumu_rewards_by_iteration = []
    cumu_rewards_by_episode = []
    
    iteration = episode = 0
    agent.reset()

    while iteration < MIN_ITERATIONS or episode < MIN_EPISODES:
        
        task.reset()
        board_state = task.grid
        reward = None
        cumulative_reward = 0

        while iteration < MIN_ITERATIONS or episode < MIN_EPISODES:
                        
            action = agent.interact(reward, board_state, iteration)

            if task.move(action, player)[1] == 50:
                print "Won!"
                break

            return_val, reward = task.move(action, player)

            if iteration < MIN_ITERATIONS:
                print np.rot90(task.grid)

                rewards_by_iteration.append(reward)
                if cumu_rewards_by_iteration == []:
                    cumu_rewards_by_iteration.append(reward)
                else:
                    cumu_rewards_by_iteration.append(cumu_rewards_by_iteration[-1] + reward)
                
            cumulative_reward += reward

            iteration += 1

        if episode < MIN_EPISODES:
            rewards_by_episode.append(cumulative_reward)
            if cumu_rewards_by_episode == []:
                cumu_rewards_by_episode.append(cumulative_reward)
            else:
                cumu_rewards_by_episode.append(cumu_rewards_by_episode[-1] + cumulative_reward)
        episode += 1
        
    return rewards_by_iteration, rewards_by_episode, cumu_rewards_by_iteration, cumu_rewards_by_episode

## Tree Data Structure and MCTS

In [12]:
class Node(object):
    """
    Define a Tree Data Structure
    """
    def __init__(self, state, parent, action_taken):
        self.state = state
        self.parent = parent
        self.actions = []
        self.action_taken = action_taken
        self.children = []
        self.total_reward = 0
        self.total_visit_count = 0.000001


In [21]:
class MCTS(object):
    """
    Monte Carlo Tree Search 
    UCT algorithm from "A Survey of Monte Carlo Tree Search Methods
    
    Node is Agent's move, next move is enemy
    """    
    def __init__(self, board, max_iter, C):
        self.board = board
        self.max_iter = max_iter
        self.C = C
        
    def reset(self):
        self.__init__(self.max_iter, self.C)
        
    def calc_next_move(self):
        return self.uct_search(self.board)
        
    def full_check_win(self,board):
        for col in xrange(0,board.grid_size):
            for row in xrange(0,board.grid_size):
                if board.check_win(col, row, 1) == 1:
                    return True
                if board.check_win(col, row, -1) == 1:
                    return True
        return False
        
    def uct_search(self,board):
#         # check for immediate win
#         for i in board.next_possible_moves():
#             b2 = copy.deepcopy(board)
#             if b2.turn() == 1:
#                 b2.move(i, 1)
#                 if self.full_check_win(b2):
#                     return i
#             if b2.turn() == -1:
#                 b2.move(i, -1)
#                 if self.full_check_win(b2):
#                     return i

#         # regular uct_search
#         iterator = 0
#         root = Node(board, None, None)
#         while iterator < self.max_iter:
#             c_node = self.tree_policy(root)
#             d = self.default_policy(c_node.state)
#             self.backup(c_node,d)
#             iterator = iterator + 1
    
        # parallel uct_search
        root = Node(board, None, None)
        for j in xrange(0,self.max_iter):
            c_node = self.tree_policy(root)
            par_iterator = [c_node for i in range(0,20)]
            pool = Pool()  # start all workers
            output = pool.map(self.parallel_uct_helper, par_iterator)
            for i in output:
                self.backup(i[0],i[1])
            
        return self.best_child(root,0)[0]
    
    def parallel_uct_helper(i):
        d = self.default_policy(i.state)
        return [i, d]        
        
    def tree_policy(self,node):
        x = node
        while self.full_check_win(x.state) == False and x.state.next_possible_moves() != []:
            if list(set(x.state.next_possible_moves()) - set(x.actions)) != []:
                return self.expand(x)
            else: 
                x = x.children[self.best_child(x,self.C)[1]]
        return x
    
    def expand(self,node):
        untried = list(set(node.state.next_possible_moves()) - set(node.actions))
        if untried != []: 
            if node.state.turn() == 1:
                child = copy.deepcopy(node)
                child.state.move(untried[0],1)
                node.children.append(Node(child.state, node, untried[0]))
                node.actions.append(untried[0])
                return node.children[-1]
            if node.state.turn() == -1:
                child = copy.deepcopy(node)
                child.state.move(untried[0],-1)
                node.children.append(Node(child.state, node, untried[0]))
                node.actions.append(untried[0])
                return node.children[-1]
        return 

    
    def best_child(self,node,c):
        child_vals = [((x.total_reward)/(x.total_visit_count) + c * np.sqrt(2*np.log(node.total_visit_count)/x.total_visit_count)) for x in node.children]
        best_inx = argmax_breaking_ties_randomly(child_vals)  
        best_c = node.children[best_inx]
        #print(child_vals)
        return best_c.action_taken, best_inx
        
    def default_policy(self,board):
        if board.turn() == 1:
            # assumes that agent is player 1
            board2 = copy.deepcopy(board)
            if self.full_check_win(board2) == True:
                return 50
            while True:                
                if board2.turn() == -1:
                    # imagined enemy                        
                    action2 = np.random.choice(board2.next_possible_moves())
                    # check win
                    for i in board2.next_possible_moves():
                        b2 = copy.deepcopy(board2)
                        b2.move(i, -1)
                        if self.full_check_win(b2):
                            action2 = i 
                    # else random
                    board2.move(action2, -1)
                    if self.full_check_win(board2) == True:
                        return -50
                    if board2.next_possible_moves() == []:
                        return 0
                # agent
                if board2.turn() == 1:
                    action = np.random.choice(board2.next_possible_moves())
                    # check win
                    for i in board2.next_possible_moves():
                        b2 = copy.deepcopy(board2)
                        b2.move(i, 1)
                        if self.full_check_win(b2):
                            action = i 
                    # else random
                    board2.move(action, 1)
                    if self.full_check_win(board2) == True:
                        return 50
                    if board2.next_possible_moves() == []:
                        return 0        
        if board.turn() == -1:
            # assumes that agent is player -1
            board2 = copy.deepcopy(board)
            if self.full_check_win(board2) == True:
                return 50
            while True:
                if board2.turn() == 1:
                    # imagined enemy
                    action2 = np.random.choice(board2.next_possible_moves())
                    # check win
                    for i in board2.next_possible_moves():
                        b2 = copy.deepcopy(board2)
                        b2.move(i, 1)
                        if self.full_check_win(b2):
                            action2 = i 
                    # else random
                    board2.move(action2, 1)
                    if self.full_check_win(board2) == True:
                        return -50
                    if board2.next_possible_moves() == []:
                        return 0
                # agent
                if board2.turn() == -1:
                    action = np.random.choice(board2.next_possible_moves())
                    # check win
                    for i in board2.next_possible_moves():
                        b2 = copy.deepcopy(board2)
                        b2.move(i, -1)
                        if self.full_check_win(b2):
                            action = i 
                    # else random
                    board2.move(action, -1)
                    if self.full_check_win(board2) == True:
                        return 50
                    if board2.next_possible_moves() == []:
                        return 0
            
    def backup(self,node,d):
        v = node 
        while v != None:
            v.total_visit_count = v.total_visit_count + 1
            v.total_reward = v.total_reward + d
            v = v.parent
        return


In [22]:
'''
Testing
'''

board = ConnectN(5,3)
board.move(4,1)
board.move(4,-1)
board.move(2,1)
board.move(1,-1)
board.move(0,1)
board.move(0,-1)
board.move(2,1)
board.move(1,-1)


MCTSAgent = MCTS(board, 1000,0.5)
print MCTSAgent.calc_next_move()
MCTSAgent.board.print_grid()
board.move(0,1)
print MCTSAgent.calc_next_move()
MCTSAgent.board.print_grid()

PicklingError: Can't pickle <type 'instancemethod'>: attribute lookup __builtin__.instancemethod failed